## Skip List

A probabilistic data-structure that allows for efficient searching, insertion, deletion, and other operations on a sorted collection of elements.

![skip-list](https://github.com/Devansh3712/tandb/assets/58616444/fa839bad-4eaf-4eba-99eb-5407689494f4)

In [5]:
import random
from typing import Tuple

class SkipListNode:
    def __init__(self, value: int, level: int) -> None:
        self.value = value
        self.forward = [None] * (level + 1)


class SkipList:
    def __init__(self, max_level: int, probability: float) -> None:
        self.max_level = max_level
        self.probability = probability
        self.header = SkipListNode(float("-inf"), max_level)
        self.level = 0

    def _random_level(self) -> int:
        level = 0
        while random.random() < self.probability and level < self.max_level:
            level += 1
        return level

    def _update_nodes(self, value: int) -> Tuple[SkipListNode, list[SkipListNode]]:
        update = [None] * (self.max_level + 1)
        current = self.header

        # Traverse the skip list from the top level to the bottom level
        for i in range(self.level, -1, -1):
            while current.forward[i] and current.forward[i].value < value:
                current = current.forward[i]
            # Keep track of nodes to be updated during insertion
            update[i] = current

        return (current, update)

    def insert(self, value: int) -> None:
        current, update = self._update_nodes(value)
        current = current.forward[0]

        if not current or current.value != value:
            new_level = self._random_level()
            if new_level > self.level:
                for i in range(self.level + 1, new_level + 1):
                    update[i] = self.header
                self.level = new_level

            new_node = SkipListNode(value, new_level)
            for i in range(new_level + 1):
                new_node.forward[i] = update[i].forward[i]
                update[i].forward[i] = new_node

    def search(self, value: int) -> bool:
        current = self.header
        for i in range(self.level, -1, -1):
            while current.forward[i] and current.forward[i].value < value:
                current = current.forward[i]

        current = current.forward[0]
        if current and current.value == value:
            return True
        return False

    def delete(self, value: int) -> None:
        current, update = self._update_nodes(value)
        current = current.forward[0]

        if current and current.value == value:
            for i in range(self.level + 1):
                if update[i].forward[i] != current:
                    break
                update[i].forward[i] = current.forward[i]
            while self.level > 0 and not self.header.forward[self.level]:
                self.level -= 1

    def print(self):
        for level in range(self.level + 1):
            print(f"L{level}: ", end="")
            node = self.header.forward[level]
            while node:
                print(node.value, end=" -> ")
                node = node.forward[level]
            print("None")

In [10]:
arr = [i for i in range(1, 11)]
s = SkipList(max_level=3, probability=0.5)

for i in arr:
    s.insert(i)

s.print()

L0: 1 -> 2 -> 3 -> 4 -> 5 -> 6 -> 7 -> 8 -> 9 -> 10 -> None
L1: 1 -> 2 -> 3 -> 4 -> 5 -> 6 -> 8 -> None
L2: 2 -> 3 -> 4 -> 6 -> 8 -> None
L3: 3 -> 4 -> 6 -> None
